# Mini-project: Gaussian Naive Bayes

## <u>Preprocessing</u>

In [1]:
# before executing:
# pip install ucimlrepo

In [2]:
import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [3]:
dataset_ids =   {
                "parkinsons": 174,
                "page-blocks": 78,
                "optical": 80,
                "musk2": 75,
                "bc-wisc-diag": 17,
                "students": 697,
                "wine": 109,
                "magic": 159,
                "balance-scale": 12,
                "glass": 42,
                "zoo": 111,
                "waveform": 107,
                "image-segmentation": 50,
                "blood": 176,
                "spect": 95,
                "yeast": 110,
                "monk": 70,
                "ecoli": 39,
                "iris": 53,
                "contraception": 30,
                "fertility": 244,
                "conn-bench-sonar":  151,
                "landsat": 146,
                "ionosphere": 52,
                "letter": 59,
                }

In [4]:
def load_dataset(id):  
    # fetch dataset 
    dataset = fetch_ucirepo(id=id) 
    
    # data (as pandas dataframes) 
    X = dataset.data.features 
    y =dataset.data.targets 
    
    # dictionary gthering infos about the metadata (url, abstract, ... etc.)
    metadata_infos_dict = dataset.metadata
    print('data url:\n', metadata_infos_dict['data_url'])
    
    # variable information
    var_infos = dataset.variables.to_numpy()
    
    data_vectors = X.to_numpy() #instance vectors with features
    features_names = X.columns.to_numpy() #getting the names of each feature
    
    data_labels = y.to_numpy() #output labels for each instance
    label_name = y.columns.to_numpy() # name of the output label
    
    return data_vectors, features_names, data_labels, label_name

In [5]:
def print_useful_data(X, X_names, y, y_name, index = 0):
    n_instances = len(X)
    n_features = len(X_names)
    
    print("number of instances: ", n_instances)
    print("number of features: ", n_features)

    print("names of the features:\n", X_names)
    print("name of the output label: ", y_name)

    print(f"instance {index} feature vector:\n", X[index])
    print(f"instance {index} output label: ", y[index])

In [6]:
def preprocessing(X, y, split=0.7):
    scaler = StandardScaler()
    try:
        #standardization
        X = scaler.fit_transform(X)
    except ValueError:
        #If non numerical data is detected, data is encoded
        X = np.array(X, dtype=object)
        encoder = OneHotEncoder()
        X_encoded = encoder.fit_transform(X).toarray()
        X = scaler.fit_transform(X_encoded)
    
    #data is split among training and testing sets
    return train_test_split(X, y, train_size=split, random_state=42)

## <u>Gaussian Naive Bayes</u>

In [7]:
class GaussianBayes:
    def __init__(self):
        self.classes = None
        self.mean = {}
        self.variance = {}
        self.prior = {}

    
    def fit(self, X_train, Y_train):
        self.classes = np.unique(Y_train)
        
        for singular_class in self.classes:
            X_c = X_train[Y_train.flatten() == singular_class]
            self.mean[singular_class] = np.mean(X_c, axis=0) 
            self.variance[singular_class] = np.var(X_c, axis=0)
            self.prior[singular_class] = X_c.shape[0] / X_train.shape[0]
            
    def gaussian_density(self, x, mean, var, epsilon=1e-9):
        # Stabilize variance to avoid division by zero
        var = var + epsilon
        return (1 / np.sqrt(2*np.pi*var)) * np.exp(-((x-mean)**2) / (2*var))
    
    def predict_sample(self, X):
        probability = []
        for singular_class in self.classes:
            log_likelihood = np.sum(np.log(np.maximum(self.gaussian_density(X, self.mean[singular_class], self.variance[singular_class]), 1e-6)))
            log_prior = np.log(self.prior[singular_class])
            probability.append(log_prior + log_likelihood)
        
        probability = np.array(probability)
        return self.classes[np.argmax(probability)]
    
    def predict(self, X_test):
        return np.array([self.predict_sample(x) for x in X_test])


In [8]:
#function to visualize clusters using PCA
def visualize_clusters(X, clusters, centroids, iteration, nb_cluster, cluster_labels=None):
    plt.figure(figsize=(8, 6))
    colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']  #define colors for clusters

    #reduce the data to 2D using PCA for visualization
    pca = PCA(n_components=2)
    X_2D = pca.fit_transform(X) 

    #if cluster assignments are provided, color by cluster membership
    if cluster_labels is not None:
        for i in range(nb_cluster):
            cluster_points_2D = X_2D[cluster_labels == i]
            plt.scatter(cluster_points_2D[:, 0], cluster_points_2D[:, 1], 
                        color=colors[i % len(colors)], label=f'Cluster {i+1}')
    else:
        for i, cluster in enumerate(clusters):
            cluster_points = np.array(cluster)
            cluster_points_2D = pca.transform(cluster_points)  #transform current cluster into 2D
            plt.scatter(cluster_points_2D[:, 0], cluster_points_2D[:, 1], 
                        color=colors[i % len(colors)], label=f'Cluster {i+1}')

    #plot centroids in 2D space
    centroids_2D = pca.transform(centroids)  #transform centroids into 2D
    plt.scatter(centroids_2D[:, 0], centroids_2D[:, 1], 
                color='black', marker='x', s=200, label='Centroids')

    plt.title(f"clusters and centroids - Iteration {iteration}")
    plt.legend()
    plt.show()

## <u>Accuracy, F1 and confusion matrice</u>

In [9]:
def calculate_metrics(true_labels, predicted_labels):
    
    accuracy = accuracy_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    cm = confusion_matrix(Y_test, Y_pred)
    
    return accuracy, f1, cm

## <u>Results on 25 datasets</u>

### Predict Students' Dropout and Academic Success

In [10]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["students"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split=split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/697/data.csv
training set size: (3096, 36)
testing set size: (1328, 36)


In [11]:
print_useful_data(X, X_names, y, y_name)

number of instances:  4424
number of features:  36
names of the features:
 ['Marital Status' 'Application mode' 'Application order' 'Course'
 'Daytime/evening attendance' 'Previous qualification'
 'Previous qualification (grade)' 'Nacionality' "Mother's qualification"
 "Father's qualification" "Mother's occupation" "Father's occupation"
 'Admission grade' 'Displaced' 'Educational special needs' 'Debtor'
 'Tuition fees up to date' 'Gender' 'Scholarship holder'
 'Age at enrollment' 'International' 'Curricular units 1st sem (credited)'
 'Curricular units 1st sem (enrolled)'
 'Curricular units 1st sem (evaluations)'
 'Curricular units 1st sem (approved)' 'Curricular units 1st sem (grade)'
 'Curricular units 1st sem (without evaluations)'
 'Curricular units 2nd sem (credited)'
 'Curricular units 2nd sem (enrolled)'
 'Curricular units 2nd sem (evaluations)'
 'Curricular units 2nd sem (approved)' 'Curricular units 2nd sem (grade)'
 'Curricular units 2nd sem (without evaluations)' 'Unemploymen

In [12]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 69.73%
F1 score: 0.68
confusion matrix:
[[305  51  85]
 [ 39  61 145]
 [ 39  43 560]]


### Wine

In [13]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["wine"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/109/data.csv
training set size: (124, 13)
testing set size: (54, 13)


In [14]:
print_useful_data(X, X_names, y, y_name)

number of instances:  178
number of features:  13
names of the features:
 ['Alcohol' 'Malicacid' 'Ash' 'Alcalinity_of_ash' 'Magnesium'
 'Total_phenols' 'Flavanoids' 'Nonflavanoid_phenols' 'Proanthocyanins'
 'Color_intensity' 'Hue' '0D280_0D315_of_diluted_wines' 'Proline']
name of the output label:  ['class']
instance 0 feature vector:
 [1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
instance 0 output label:  [1]


In [15]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 98.15%
F1 score: 0.98
confusion matrix:
[[19  0  0]
 [ 0 21  0]
 [ 0  1 13]]


### MAGIC gamma telescope

In [16]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["magic"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/159/data.csv
training set size: (13314, 10)
testing set size: (5706, 10)


In [17]:
print_useful_data(X, X_names, y, y_name)

number of instances:  19020
number of features:  10
names of the features:
 ['fLength' 'fWidth' 'fSize' 'fConc' 'fConc1' 'fAsym' 'fM3Long' 'fM3Trans'
 'fAlpha' 'fDist']
name of the output label:  ['class']
instance 0 feature vector:
 [28.7967 16.0021  2.6449  0.3918  0.1982 27.7004 22.011  -8.2027 40.092
 81.8828]
instance 0 output label:  ['g']


In [18]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 72.75%
F1 score: 0.70
confusion matrix:
[[3410  295]
 [1260  741]]


### Parkinsons

In [19]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["parkinsons"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/174/data.csv
training set size: (136, 22)
testing set size: (59, 22)


In [20]:
print_useful_data(X, X_names, y, y_name)

number of instances:  195
number of features:  22
names of the features:
 ['MDVP:Fo' 'MDVP:Fhi' 'MDVP:Flo' 'MDVP:Jitter' 'MDVP:Jitter' 'MDVP:RAP'
 'MDVP:PPQ' 'Jitter:DDP' 'MDVP:Shimmer' 'MDVP:Shimmer' 'Shimmer:APQ3'
 'Shimmer:APQ5' 'MDVP:APQ' 'Shimmer:DDA' 'NHR' 'HNR' 'RPDE' 'DFA'
 'spread1' 'spread2' 'D2' 'PPE']
name of the output label:  ['status']
instance 0 feature vector:
 [ 1.199920e+02  1.573020e+02  7.499700e+01  7.840000e-03  7.840000e-03
  3.700000e-03  5.540000e-03  1.109000e-02  4.374000e-02  4.374000e-02
  2.182000e-02  3.130000e-02  2.971000e-02  6.545000e-02  2.211000e-02
  2.103300e+01  4.147830e-01  8.152850e-01 -4.813031e+00  2.664820e-01
  2.301442e+00  2.846540e-01]
instance 0 output label:  [1]


In [21]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 76.27%
F1 score: 0.78
confusion matrix:
[[12  3]
 [11 33]]


### Page-blocks

In [22]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["page-blocks"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/78/data.csv
training set size: (3831, 10)
testing set size: (1642, 10)


In [23]:
print_useful_data(X, X_names, y, y_name)

number of instances:  5473
number of features:  10
names of the features:
 ['height' 'length' 'area' 'eccen' 'p_black' 'p_and' 'mean_tr' 'blackpix'
 'blackand' 'wb_trans']
name of the output label:  ['class']
instance 0 feature vector:
 [ 5.     7.    35.     1.4    0.4    0.657  2.33  14.    23.     6.   ]
instance 0 output label:  [1]


In [24]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 87.27%
F1 score: 0.90
confusion matrix:
[[1311    7    8  118   22]
 [  24   79    1    2    0]
 [   2    0    7    0    1]
 [   0    2    0   28    1]
 [  12    0    5    4    8]]


### Optical

In [25]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["optical"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)

num_sample = 6000
X_train, X_test = X_train[:num_sample*2,:], X_test[:num_sample,:]
Y_train, Y_test = Y_train[:num_sample*2], Y_test[:num_sample]

print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/80/data.csv
training set size: (3933, 64)
testing set size: (1687, 64)


In [26]:
print_useful_data(X, X_names, y, y_name)

number of instances:  5620
number of features:  64
names of the features:
 ['Attribute1' 'Attribute2' 'Attribute3' 'Attribute4' 'Attribute5'
 'Attribute6' 'Attribute7' 'Attribute8' 'Attribute9' 'Attribute10'
 'Attribute11' 'Attribute12' 'Attribute13' 'Attribute14' 'Attribute15'
 'Attribute16' 'Attribute17' 'Attribute18' 'Attribute19' 'Attribute20'
 'Attribute21' 'Attribute22' 'Attribute23' 'Attribute24' 'Attribute25'
 'Attribute26' 'Attribute27' 'Attribute28' 'Attribute29' 'Attribute30'
 'Attribute31' 'Attribute32' 'Attribute33' 'Attribute34' 'Attribute35'
 'Attribute36' 'Attribute37' 'Attribute38' 'Attribute39' 'Attribute40'
 'Attribute41' 'Attribute42' 'Attribute43' 'Attribute44' 'Attribute45'
 'Attribute46' 'Attribute47' 'Attribute48' 'Attribute49' 'Attribute50'
 'Attribute51' 'Attribute52' 'Attribute53' 'Attribute54' 'Attribute55'
 'Attribute56' 'Attribute57' 'Attribute58' 'Attribute59' 'Attribute60'
 'Attribute61' 'Attribute62' 'Attribute63' 'Attribute64']
name of the output label

In [27]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 61.94%
F1 score: 0.60
confusion matrix:
[[170   0   0   0   0   0   0   0   0   0]
 [  0 116   0   0   0   0  14   0  38   5]
 [  2  12  66   0   0   0   0   0  74   0]
 [  7   0   0  50   0   0   0   8 104   4]
 [ 21  24   0   0  43   0  37  49   6   3]
 [ 21   1   0   0   0  46   1   8  74   2]
 [  1   1   0   0   0   0 166   0   0   0]
 [  0   3   0   0   0   0   0 180   3   0]
 [  3   2   0   0   0   0   0   1 147   0]
 [ 38   6   0   0   0   0   2  21  46  61]]


### Ionosphere

In [28]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["ionosphere"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/52/data.csv
training set size: (245, 34)
testing set size: (106, 34)


In [29]:
print_useful_data(X, X_names, y, y_name)

number of instances:  351
number of features:  34
names of the features:
 ['Attribute1' 'Attribute2' 'Attribute3' 'Attribute4' 'Attribute5'
 'Attribute6' 'Attribute7' 'Attribute8' 'Attribute9' 'Attribute10'
 'Attribute11' 'Attribute12' 'Attribute13' 'Attribute14' 'Attribute15'
 'Attribute16' 'Attribute17' 'Attribute18' 'Attribute19' 'Attribute20'
 'Attribute21' 'Attribute22' 'Attribute23' 'Attribute24' 'Attribute25'
 'Attribute26' 'Attribute27' 'Attribute28' 'Attribute29' 'Attribute30'
 'Attribute31' 'Attribute32' 'Attribute33' 'Attribute34']
name of the output label:  ['Class']
instance 0 feature vector:
 [ 1.       0.       0.99539 -0.05889  0.85243  0.02306  0.83398 -0.37708
  1.       0.0376   0.85243 -0.17755  0.59755 -0.44945  0.60536 -0.38223
  0.84356 -0.38542  0.58212 -0.32192  0.56971 -0.29674  0.36946 -0.47357
  0.56811 -0.51171  0.41078 -0.46168  0.21266 -0.3409   0.42267 -0.54487
  0.18641 -0.453  ]
instance 0 output label:  ['g']


In [30]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 88.68%
F1 score: 0.88
confusion matrix:
[[28 11]
 [ 1 66]]


### Glass Identification

In [31]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["glass"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/42/data.csv
training set size: (149, 9)
testing set size: (65, 9)


In [32]:
print_useful_data(X, X_names, y, y_name)

number of instances:  214
number of features:  9
names of the features:
 ['RI' 'Na' 'Mg' 'Al' 'Si' 'K' 'Ca' 'Ba' 'Fe']
name of the output label:  ['Type_of_glass']
instance 0 feature vector:
 [1.52101e+00 1.36400e+01 4.49000e+00 1.10000e+00 7.17800e+01 6.00000e-02
 8.75000e+00 0.00000e+00 0.00000e+00]
instance 0 output label:  [1]


In [33]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 21.54%
F1 score: 0.21
confusion matrix:
[[ 0  1 18  0  0  0]
 [ 1  3 15  0  4  0]
 [ 1  0  3  0  0  0]
 [ 0  3  0  1  2  0]
 [ 0  0  0  0  3  0]
 [ 0  0  0  1  5  4]]


### Letter Recognition

In [34]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["letter"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/59/data.csv
training set size: (14000, 16)
testing set size: (6000, 16)


In [35]:
print_useful_data(X, X_names, y, y_name)

number of instances:  20000
number of features:  16
names of the features:
 ['x-box' 'y-box' 'width' 'high' 'onpix' 'x-bar' 'y-bar' 'x2bar' 'y2bar'
 'xybar' 'x2ybr' 'xy2br' 'x-ege' 'xegvy' 'y-ege' 'yegvx']
name of the output label:  ['lettr']
instance 0 feature vector:
 [ 2  8  3  5  1  8 13  0  6  6 10  8  0  8  0  8]
instance 0 output label:  ['T']


In [36]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 64.30%
F1 score: 0.64
confusion matrix:
[[213   0   0   0   0   0   0   1   0   0   1   0   6   1   0   0   1   1
    7   0   0   0   0   0   1   0]
 [  0 155   0   6   0   0   2   5  20   0   5   0   4   0   2   0   4  16
    5   0   0   0   4   1   0   0]
 [  0   0 144   0   6   1  16   1   0   0  11   0   2   0   6   0   4   0
    5   2   2   1   0   0   0   0]
 [  2  13   0 193   0   0   0   5   3   5   5   0   3   0   3   1   0  11
    4   0   0   0   1   1   0   0]
 [  0   5   1   0  84   0  43   0  22   0   9   0   0   0   0   0  12   0
   19   4   0   0   0  33   2   4]
 [  0   9   0   9   0 152   9   1   1   0   0   0   0   4   0   9   5   1
    4   4   0   0   2   0   1   0]
 [  5   6  50   1   1   0 127   0   4   0   2   0   4   0   2   0  12   4
    4   0   0   0   6   2   0   0]
 [  1   6   0  19   0   0   3  67   4   0   9   0   5   2  35   0   1  20
    1   0   8   0   0  32   5   0]
 [  0   4   0  22   2   0   0   0 164   7   0   2   0   0   0   1   3   0
   1

### Connectionist Bench (Sonar, Mines vs. Rocks)

In [37]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["conn-bench-sonar"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/151/data.csv
training set size: (145, 60)
testing set size: (63, 60)


In [38]:
print_useful_data(X, X_names, y, y_name)

number of instances:  208
number of features:  60
names of the features:
 ['Attribute1' 'Attribute2' 'Attribute3' 'Attribute4' 'Attribute5'
 'Attribute6' 'Attribute7' 'Attribute8' 'Attribute9' 'Attribute10'
 'Attribute11' 'Attribute12' 'Attribute13' 'Attribute14' 'Attribute15'
 'Attribute16' 'Attribute17' 'Attribute18' 'Attribute19' 'Attribute20'
 'Attribute21' 'Attribute22' 'Attribute23' 'Attribute24' 'Attribute25'
 'Attribute26' 'Attribute27' 'Attribute28' 'Attribute29' 'Attribute30'
 'Attribute31' 'Attribute32' 'Attribute33' 'Attribute34' 'Attribute35'
 'Attribute36' 'Attribute37' 'Attribute38' 'Attribute39' 'Attribute40'
 'Attribute41' 'Attribute42' 'Attribute43' 'Attribute44' 'Attribute45'
 'Attribute46' 'Attribute47' 'Attribute48' 'Attribute49' 'Attribute50'
 'Attribute51' 'Attribute52' 'Attribute53' 'Attribute54' 'Attribute55'
 'Attribute56' 'Attribute57' 'Attribute58' 'Attribute59' 'Attribute60']
name of the output label:  ['class']
instance 0 feature vector:
 [0.02   0.0371 0.

In [39]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 77.78%
F1 score: 0.78
confusion matrix:
[[23 12]
 [ 2 26]]


### Musk (Version 2)

In [40]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["musk2"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/75/data.csv
training set size: (4618, 166)
testing set size: (1980, 166)


In [41]:
print_useful_data(X, X_names, y, y_name)

number of instances:  6598
number of features:  166
names of the features:
 ['f1' 'f2' 'f3' 'f4' 'f5' 'f6' 'f7' 'f8' 'f9' 'f10' 'f11' 'f12' 'f13'
 'f14' 'f15' 'f16' 'f17' 'f18' 'f19' 'f20' 'f21' 'f22' 'f23' 'f24' 'f25'
 'f26' 'f27' 'f28' 'f29' 'f30' 'f31' 'f32' 'f33' 'f34' 'f35' 'f36' 'f37'
 'f38' 'f39' 'f40' 'f41' 'f42' 'f43' 'f44' 'f45' 'f46' 'f47' 'f48' 'f49'
 'f50' 'f51' 'f52' 'f53' 'f54' 'f55' 'f56' 'f57' 'f58' 'f59' 'f60' 'f61'
 'f62' 'f63' 'f64' 'f65' 'f66' 'f67' 'f68' 'f69' 'f70' 'f71' 'f72' 'f73'
 'f74' 'f75' 'f76' 'f77' 'f78' 'f79' 'f80' 'f81' 'f82' 'f83' 'f84' 'f85'
 'f86' 'f87' 'f88' 'f89' 'f90' 'f91' 'f92' 'f93' 'f94' 'f95' 'f96' 'f97'
 'f98' 'f99' 'f100' 'f101' 'f102' 'f103' 'f104' 'f105' 'f106' 'f107'
 'f108' 'f109' 'f110' 'f111' 'f112' 'f113' 'f114' 'f115' 'f116' 'f117'
 'f118' 'f119' 'f120' 'f121' 'f122' 'f123' 'f124' 'f125' 'f126' 'f127'
 'f128' 'f129' 'f130' 'f131' 'f132' 'f133' 'f134' 'f135' 'f136' 'f137'
 'f138' 'f139' 'f140' 'f141' 'f142' 'f143' 'f144' 'f145' 'f14

In [42]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 75.15%
F1 score: 0.78
confusion matrix:
[[1240  433]
 [  59  248]]


### Breast Cancer Wisconsin (Diagnostic)

In [43]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["bc-wisc-diag"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/17/data.csv
training set size: (398, 30)
testing set size: (171, 30)


In [44]:
print_useful_data(X, X_names, y, y_name)

number of instances:  569
number of features:  30
names of the features:
 ['radius1' 'texture1' 'perimeter1' 'area1' 'smoothness1' 'compactness1'
 'concavity1' 'concave_points1' 'symmetry1' 'fractal_dimension1' 'radius2'
 'texture2' 'perimeter2' 'area2' 'smoothness2' 'compactness2' 'concavity2'
 'concave_points2' 'symmetry2' 'fractal_dimension2' 'radius3' 'texture3'
 'perimeter3' 'area3' 'smoothness3' 'compactness3' 'concavity3'
 'concave_points3' 'symmetry3' 'fractal_dimension3']
name of the output label:  ['Diagnosis']
instance 0 feature vector:
 [1.799e+01 1.038e+01 1.228e+02 1.001e+03 1.184e-01 2.776e-01 3.001e-01
 1.471e-01 2.419e-01 7.871e-02 1.095e+00 9.053e-01 8.589e+00 1.534e+02
 6.399e-03 4.904e-02 5.373e-02 1.587e-02 3.003e-02 6.193e-03 2.538e+01
 1.733e+01 1.846e+02 2.019e+03 1.622e-01 6.656e-01 7.119e-01 2.654e-01
 4.601e-01 1.189e-01]
instance 0 output label:  ['M']


In [45]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 93.57%
F1 score: 0.94
confusion matrix:
[[103   5]
 [  6  57]]


### Balance-scale

In [46]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["balance-scale"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/12/data.csv
training set size: (437, 4)
testing set size: (188, 4)


In [47]:
print_useful_data(X, X_names, y, y_name)

number of instances:  625
number of features:  4
names of the features:
 ['right-distance' 'right-weight' 'left-distance' 'left-weight']
name of the output label:  ['class']
instance 0 feature vector:
 [1 1 1 1]
instance 0 output label:  ['B']


In [48]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 88.83%
F1 score: 0.84
confusion matrix:
[[ 0 13  5]
 [ 0 80  0]
 [ 0  3 87]]


### Contraception

In [49]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["contraception"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/30/data.csv
training set size: (1031, 9)
testing set size: (442, 9)


In [50]:
print_useful_data(X, X_names, y, y_name)

number of instances:  1473
number of features:  9
names of the features:
 ['wife_age' 'wife_edu' 'husband_edu' 'num_children' 'wife_religion'
 'wife_working' 'husband_occupation' 'standard_of_living_index'
 'media_exposure']
name of the output label:  ['contraceptive_method']
instance 0 feature vector:
 [24  2  3  3  1  1  2  3  0]
instance 0 output label:  [1]


In [51]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 45.25%
F1 score: 0.45
confusion matrix:
[[77 53 64]
 [13 55 33]
 [42 37 68]]


### Fertility

In [52]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["fertility"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/244/data.csv
training set size: (70, 9)
testing set size: (30, 9)


In [53]:
print_useful_data(X, X_names, y, y_name)

number of instances:  100
number of features:  9
names of the features:
 ['season' 'age' 'child_diseases' 'accident' 'surgical_intervention'
 'high_fevers' 'alcohol' 'smoking' 'hrs_sitting']
name of the output label:  ['diagnosis']
instance 0 feature vector:
 [-0.33  0.69  0.    1.    1.    0.    0.8   0.    0.88]
instance 0 output label:  ['N']


In [54]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 93.33%
F1 score: 0.92
confusion matrix:
[[27  0]
 [ 2  1]]


### Waveform (Version 1)

In [55]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["waveform"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/107/data.csv
training set size: (3500, 21)
testing set size: (1500, 21)


In [56]:
print_useful_data(X, X_names, y, y_name)

number of instances:  5000
number of features:  21
names of the features:
 ['Attribute1' 'Attribute2' 'Attribute3' 'Attribute4' 'Attribute5'
 'Attribute6' 'Attribute7' 'Attribute8' 'Attribute9' 'Attribute10'
 'Attribute11' 'Attribute12' 'Attribute13' 'Attribute14' 'Attribute15'
 'Attribute16' 'Attribute17' 'Attribute18' 'Attribute19' 'Attribute20'
 'Attribute21']
name of the output label:  ['class']
instance 0 feature vector:
 [-1.23 -1.56 -1.75 -0.28  0.6   2.22  0.85  0.21 -0.2   0.89  1.08  4.2
  2.89  7.75  4.59  3.15  5.12  3.32  1.2   0.24 -0.56]
instance 0 output label:  [2]


In [57]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 80.87%
F1 score: 0.80
confusion matrix:
[[262 117 107]
 [  5 490  24]
 [  3  31 461]]


### Image Segmentation

In [58]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["image-segmentation"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/50/data.csv
training set size: (147, 19)
testing set size: (63, 19)


In [59]:
print_useful_data(X, X_names, y, y_name)

number of instances:  210
number of features:  19
names of the features:
 ['region-centroid-col' 'region-centroid-row' 'region-pixel-count'
 'short-line-density-5' 'short-line-density-2' 'vedge-mean' 'vedge-sd'
 'hedge-mean' 'hedge-sd' 'intensity-mean' 'rawred-mean' 'rawblue-mean'
 'rawgreen-mean' 'exred-mean' 'exblue-mean' 'exgreen-mean' 'value-mean'
 'saturation-mean' 'hue-mean']
name of the output label:  ['class']
instance 0 feature vector:
 [ 1.4000000e+02  1.2500000e+02  9.0000000e+00  0.0000000e+00
  0.0000000e+00  2.7777790e-01  6.2963010e-02  6.6666675e-01
  3.1111118e-01  6.1851850e+00  7.3333335e+00  7.6666665e+00
  3.5555556e+00  3.4444444e+00  4.4444447e+00 -7.8888890e+00
  7.7777777e+00  5.4563490e-01 -1.1218182e+00]
instance 0 output label:  ['BRICKFACE']


In [60]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 66.67%
F1 score: 0.65
confusion matrix:
[[ 3  0  0  0  0  0  6]
 [ 1  6  0  0  0  0  3]
 [ 0  1  1  0  0  0  9]
 [ 0  0  0 10  0  0  0]
 [ 0  0  0  0  9  0  0]
 [ 0  0  0  0  0  5  0]
 [ 0  0  1  0  0  0  8]]


### Blood Transfusion Service

In [61]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["blood"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/176/data.csv
training set size: (523, 4)
testing set size: (225, 4)


In [62]:
print_useful_data(X, X_names, y, y_name)

number of instances:  748
number of features:  4
names of the features:
 ['Recency' 'Frequency' 'Monetary' 'Time']
name of the output label:  ['Donated_Blood']
instance 0 feature vector:
 [    2    50 12500    98]
instance 0 output label:  [1]


In [63]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 74.67%
F1 score: 0.68
confusion matrix:
[[160   5]
 [ 52   8]]


### SPECT Heart

In [64]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["spect"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/95/data.csv
training set size: (186, 22)
testing set size: (81, 22)


In [65]:
print_useful_data(X, X_names, y, y_name)

number of instances:  267
number of features:  22
names of the features:
 ['F1' 'F2' 'F3' 'F4' 'F5' 'F6' 'F7' 'F8' 'F9' 'F10' 'F11' 'F12' 'F13'
 'F14' 'F15' 'F16' 'F17' 'F18' 'F19' 'F20' 'F21' 'F22']
name of the output label:  ['OVERALL_DIAGNOSIS']
instance 0 feature vector:
 [0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0]
instance 0 output label:  [1]


In [66]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 55.56%
F1 score: 0.59
confusion matrix:
[[15  0]
 [36 30]]


### Statlog (Landsat Satellite)

In [67]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["landsat"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/146/data.csv
training set size: (4504, 36)
testing set size: (1931, 36)


In [68]:
print_useful_data(X, X_names, y, y_name)

number of instances:  6435
number of features:  36
names of the features:
 ['Attribute1' 'Attribute2' 'Attribute3' 'Attribute4' 'Attribute5'
 'Attribute6' 'Attribute7' 'Attribute8' 'Attribute9' 'Attribute10'
 'Attribute11' 'Attribute12' 'Attribute13' 'Attribute14' 'Attribute15'
 'Attribute16' 'Attribute17' 'Attribute18' 'Attribute19' 'Attribute20'
 'Attribute21' 'Attribute22' 'Attribute23' 'Attribute24' 'Attribute25'
 'Attribute26' 'Attribute27' 'Attribute28' 'Attribute29' 'Attribute30'
 'Attribute31' 'Attribute32' 'Attribute33' 'Attribute34' 'Attribute35'
 'Attribute36']
name of the output label:  ['class']
instance 0 feature vector:
 [ 92 115 120  94  84 102 106  79  84 102 102  83 101 126 133 103  92 112
 118  85  84 103 104  81 102 126 134 104  88 121 128 100  84 107 113  87]
instance 0 output label:  [3]


In [69]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 79.13%
F1 score: 0.80
confusion matrix:
[[351   0  14   0  85   0]
 [  5 165   0   0  15   1]
 [  6   0 367  40   2   1]
 [  2   0  27 140   3  29]
 [ 21   1   0  11 161  25]
 [  0   0   2  94  19 344]]


### Yeast

In [70]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["yeast"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/110/data.csv
training set size: (1038, 8)
testing set size: (446, 8)


In [71]:
print_useful_data(X, X_names, y, y_name)

number of instances:  1484
number of features:  8
names of the features:
 ['mcg' 'gvh' 'alm' 'mit' 'erl' 'pox' 'vac' 'nuc']
name of the output label:  ['localization_site']
instance 0 feature vector:
 [0.58 0.61 0.47 0.13 0.5  0.   0.48 0.22]
instance 0 output label:  ['MIT']


In [72]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 15.25%
F1 score: 0.18
confusion matrix:
[[  0   1   7   1   0   0   2  18   0 116]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   8   0   0   0   0   0   0   2]
 [  0   0   6   9   0   0   0   0   0   0]
 [  0   0   3   4   0   0   0   0   0   4]
 [  0   0   0   0   0   1   0   4   0  46]
 [  0   0  10   3   0   0  16   2   0  43]
 [  0   0   3   1   0   0   3  26   0  93]
 [  0   0   1   0   0   0   0   0   3   0]
 [  0   0   3   2   0   0   0   0   0   5]]


### Monk

In [73]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["monk"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/70/data.csv
training set size: (302, 6)
testing set size: (130, 6)


In [74]:
print_useful_data(X, X_names, y, y_name)

number of instances:  432
number of features:  6
names of the features:
 ['a1' 'a2' 'a3' 'a4' 'a5' 'a6']
name of the output label:  ['class']
instance 0 feature vector:
 [1 1 1 1 1 1]
instance 0 output label:  [1]


In [75]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 71.54%
F1 score: 0.72
confusion matrix:
[[47 14]
 [23 46]]


### Ecoli

In [76]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["ecoli"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/39/data.csv
training set size: (235, 7)
testing set size: (101, 7)


In [77]:
print_useful_data(X, X_names, y, y_name)

number of instances:  336
number of features:  7
names of the features:
 ['mcg' 'gvh' 'lip' 'chg' 'aac' 'alm1' 'alm2']
name of the output label:  ['class']
instance 0 feature vector:
 [0.49 0.29 0.48 0.5  0.56 0.24 0.35]
instance 0 output label:  ['cp']


In [78]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 73.27%
F1 score: 0.67
confusion matrix:
[[44  1  0  0  0  0  1]
 [ 2 17  0  1  0  0  0]
 [ 0  0  0  0  0  0  1]
 [ 0 11  0  0  0  0  0]
 [ 0  0  0  0  0  0  7]
 [ 0  0  0  0  0  1  0]
 [ 1  2  0  0  0  0 12]]


### Iris

In [79]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["iris"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/53/data.csv
training set size: (105, 4)
testing set size: (45, 4)


In [80]:
print_useful_data(X, X_names, y, y_name)

number of instances:  150
number of features:  4
names of the features:
 ['sepal length' 'sepal width' 'petal length' 'petal width']
name of the output label:  ['class']
instance 0 feature vector:
 [5.1 3.5 1.4 0.2]
instance 0 output label:  ['Iris-setosa']


In [81]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 97.78%
F1 score: 0.98
confusion matrix:
[[19  0  0]
 [ 0 12  1]
 [ 0  0 13]]


### Zoo

In [82]:
X, X_names, y, y_name = load_dataset(id = dataset_ids["zoo"])
split_size = 0.7
X_train, X_test, Y_train, Y_test = preprocessing(X, y, split_size)
print(f'training set size: {X_train.shape}')
print(f'testing set size: {X_test.shape}')

data url:
 https://archive.ics.uci.edu/static/public/111/data.csv
training set size: (70, 16)
testing set size: (31, 16)


In [83]:
print_useful_data(X, X_names, y, y_name)

number of instances:  101
number of features:  16
names of the features:
 ['hair' 'feathers' 'eggs' 'milk' 'airborne' 'aquatic' 'predator' 'toothed'
 'backbone' 'breathes' 'venomous' 'fins' 'legs' 'tail' 'domestic'
 'catsize']
name of the output label:  ['type']
instance 0 feature vector:
 [1 0 0 1 0 0 1 1 1 1 0 0 4 0 0 1]
instance 0 output label:  [1]


In [84]:
#model training
model = GaussianBayes()
model.fit(X_train, Y_train)

#labels prediction
Y_pred = model.predict(X_test)

accuracy, f1, cm = calculate_metrics(Y_test, Y_pred)
print(f"accuracy: {accuracy * 100:.2f}%")
print(f"F1 score: {f1:.2f}")
print(f"confusion matrix:\n{cm}")

accuracy: 90.32%
F1 score: 0.91
confusion matrix:
[[14  0  0  1  0  0  0]
 [ 0  3  0  0  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  0  2  0  0  0]
 [ 0  0  0  0  2  0  0]
 [ 0  0  0  0  0  5  0]
 [ 0  0  1  0  0  0  2]]
